In [3]:
using Oscar

In [4]:
cd("..")

In [5]:
currentDir = pwd() # make sure you are running from the main directory.
include(joinpath(currentDir, "src/fileHandling.jl"))
include(joinpath(currentDir, "src/isolate3Lines.jl"))
include(joinpath(currentDir, "src/matroid_realization.jl"))
include(joinpath(currentDir, "src/Jacobian_Criterion.jl"))

realization_space_2_singular_locus (generic function with 1 method)

First we isolate those that satisfy the three lines property, i.e., every element of the ground set is contained in at least 3 lines (cyclic flats of rank 2). There are **151** such matroids.

In [ ]:
db = Polymake.Polydb.get_db();
collection = db["Matroids.Small"];
d3n10 = Polymake.Polydb.find(collection, Dict("RANK" => 3, "SIMPLE"=>true, "N_ELEMENTS"=>10));


n3C10 = subsets(collect(1:10), 3);
n3C10 = sort(n3C10, by =  x-> reverse(x));

lines_3 = []
for c in d3n10
    Q = Matroid(c)
    ns = count_3_lines_thru_all_points(Q)
    if length(ns) == 0
        continue
    end    
    if minimum(ns) >= 3
        push!(lines_3, to_revlex(Q, n3C10))
    end
end

This data is precomputed in `d3n10/3lines_d3n10.dat`.

In [111]:
lines_3_precomputed = vec(readlines(joinpath(currentDir, "d3n10/3lines_d3n10.dat")));

In [ ]:
Set(lines_3) == Set(lines_3_precomputed)

The file `d3n10/not_realizable.dat` contains those matroids in `d3n10/3lines_d3n10.dat` that are not realizable over $\mathbb{Q}$. There are **44** such matroids.

In [112]:
not_realizable = vec(readlines(joinpath(currentDir, "d3n10/nonrealizable.dat")));

In [ ]:
not_realizable = vec(readlines(joinpath(currentDir, "d3n10/nonrealizable.dat")));

test_nonrealizable = []

for Qstr in not_realizable
    Q = matroid_from_revlex_basis_encoding(Qstr, 3, 10) 
    As = rank_plus1_circuits(Q) 
    MR = new_matroid_realization_space(Q, As[1]; F=QQ, saturate=true)
    push!(test_nonrealizable, !MR.representable)
end

In [ ]:
all(test_nonrealizable)

The files `d3n10/zero_ideal.dat`, `d3n10/univariate_ideal.dat`, and `d3n10/principal_ideal.dat` record matroids and a maximal circuit. A pair of matroid and circuit is recorded as a `*/0` vector. The first 10 entries record the circuit (e.g., `**0*00*000` is the circuit `[1,2,4,7]`) and the rest record the matroid in the usual revlex basis encoding. 

**Note:** One could check that these are smooth using the `singular_locus` function in `src/Jacobian_Criterion.jl`. To make this computation more transparent, we carry out the smoothness check explicitly.

The files `d3n10/zero_ideal.dat` contains those matroids in `d3n10/3lines_d3n10.dat` and a maximal circuit such that the ideal of the realization space is zero. Therefore these realization spaces are smooth. There are **46** such matroids.

In [113]:
zero_ideal = vec(readlines(joinpath(currentDir, "d3n10/zero_ideal.dat")));

In [ ]:
test_zero = []
for cir_Qstr in zero_ideal
    A = [i for i in 1:10 if string(cir_Qstr[i]) == "*"]     
    Qstr = cir_Qstr[11:130]
    Q = matroid_from_revlex_basis_encoding(Qstr, 3, 10) 
    MR = new_matroid_realization_space(Q, A; F=QQ, saturate=true)
        
    MR = reduce_ideal_full(MR)
    I = MR.defining_ideal
    push!(test_zero, iszero(I))
end

In [ ]:
all(test_zero)

The files `d3n10/univariate_ideal.dat` contains those matroids in `d3n10/3lines_d3n10.dat` and a maximal circuit such that the ideal of the realization space is univariate. These realization spaces are smooth provided the discriminant is nonzero.  There are **33** such matroids.

In [114]:
univariate_ideal = vec(readlines(joinpath(currentDir, "d3n10/univariate_ideal.dat")));

In [ ]:
test_univariate = []
for cir_Qstr in univariate_ideal
    A = [i for i in 1:10 if string(cir_Qstr[i]) == "*"]     
    Qstr = cir_Qstr[11:130]
    Q = matroid_from_revlex_basis_encoding(Qstr, 3, 10) 
    MR = new_matroid_realization_space(Q, A; F=QQ, saturate=true)
        
    MR = reduce_ideal_full(MR)
    I = MR.defining_ideal
    length_vs = length(ideal_vars(gens(I))) 
    push!(test_univariate, isone(length_vs))
end

In [ ]:
all(test_univariate)

The files `d3n10/principal_ideal.dat` contains those matroids in `d3n10/3lines_d3n10.dat` and a maximal circuit such that the ideal of the realization space is principal. There are **28** such matroids. By the Jacobian criterion, the affine scheme $\mathsf{Spec}(S^{-1}\mathbb{Q}[x_1,\ldots,x_n] / \langle f \rangle)$ is smooth if and only if the ideal

$$
J = \langle f, \frac{\partial f}{\partial x_1}, \ldots, \frac{\partial f}{\partial x_n}\rangle
$$

is the unit ideal in $S^{-1} \mathbb{Q}[x_1,\ldots,x_n]$. This is equivalent to the saturation $(J:S^{\infty})$ being the unit ideal in $\mathbb{Q}[x_1,\ldots,x_n]$. We perform this explicit check here. 

In [115]:
principal_ideal = vec(readlines(joinpath(currentDir, "d3n10/principal_ideal.dat")));

In [21]:
not_realizable = []
test_principal = []
for cir_Qstr in principal_ideal
    A = [i for i in 1:10 if string(cir_Qstr[i]) == "*"]     
    Qstr = cir_Qstr[11:130]
    Q = matroid_from_revlex_basis_encoding(Qstr, 3, 10) 
    MR = new_matroid_realization_space(Q, A; F=QQ, saturate=true)
        
    MR = reduce_ideal_full(MR)
    R = MR.ambient_ring
    x = gens(R)
    I = MR.defining_ideal
    Igens = gens(I)
    length(Igens) != 1 && error("not principal") 
    JM = jacobian_matrix(R, x, Igens)
    nr, nc = size(JM) 
    J = I + ideal(R, [JM[1,c] for c in 1:nc])
    Sing = stepwise_saturation(J, MR.inequations)
    #Sing = realization_space_2_singular_locus(MR)
    push!(test_principal, isone(Sing))
end

In [20]:
all(test_principal)

true

## Example

Let $\mathsf{Q}$ be the matroid whose lines are


$$
\mathcal{L}(\mathsf{Q}) = \{\{1,2,10\},\{1,3,5\},\{1,4,8\},\{2,3,6\},\{2,4,7\},\{2,5,8\},\{3,4,9\},\{3,7,10\},\{5,6,7\},\{5,9,10\},\{6,8,9\} \}
$$



In [70]:
NB = [[1,2,10],[1,3,5],[1,4,8],[2,3,6],[2,4,7],[2,5,8],[3,4,9],[3,7,10],[5,6,7],[5,9,10],[6,8,9]]
A = [1,2,3,4]
Q = matroid_from_nonbases(NB,10)

Matroid of rank 3 on 10 elements

In [71]:
MR = new_matroid_realization_space(Q, A; F=QQ, saturate=true)

[1   0   0   1   x1    0   x3   x5   x7   x9]
[0   1   0   1    0   x2   x4   x6   x8    1]
[0   0   1   1    1    1    1    1    1    0]

The representations of the matroid are parametrized by the matrix
in the Multivariate Polynomial Ring in 9 variables x1, x2, x3, x4, ..., x9 over Rational Field
within the vanishing set of the ideal
ideal(x7 - x8, x6 - 1, x3 - 1, x1 - x5, x5 + x8*x9 - x8, x4*x9 - 1, x2*x9 + x5 - 1, x2 + x4*x8 - x4 - x8, x2*x8 - x4 + x5*x8 - x8, x5^2 - x5 + 1, x2 + x4*x5 - x4, x2*x5 - x4, x2^2 - x2*x4 + x4^2)
avoiding the zero loci of the polynomials
RingElem[x9, x7 - x8*x9, x7 - x8*x9 + x9 - 1, x2*x9 + x7 - x8*x9, x3 - x4*x9 - x7 + x8*x9, x5 - x6*x9 - x7 + x8*x9, x5 - x6*x9, x5 - x6*x9 + x9 - 1, x1 - x5 + x6*x9, x2*x9 + x5 - x6*x9  …  x3, x2*x3 - x2 - x3 + x4, x4, x1 - x3, x1*x4 - x1 + x3 - x4, x4 - 1, x3 - x4, x1*x2 - x1 - x2, x2 - 1, x1 - 1]


In [72]:
MR2 = reduce_ideal_full(MR)

[1   0   0   1   -x1*x2 + x1                0   x2   -x1*x2 + x1   x1   x2]
[0   1   0   1             0   x1*x2 - x1 + 1    1             1   x1    1]
[0   0   1   1             1               x2   x2             1    1    0]

The representations of the matroid are parametrized by the matrix
in the Multivariate Polynomial Ring in x1, x2 over Rational Field
within the vanishing set of the ideal
ideal(x1^2*x2^2 - 2*x1^2*x2 + x1^2 + x1*x2 - x1 + 1)
avoiding the zero loci of the polynomials
RingElem[x2, x1, x2 - 1, x1 - 1, x1*x2 - x1 + x2, x1*x2 - x1 + 1, x1*x2 - x1 - x2 + 2, x1^2*x2 - x1^2 + x1 - 1, x1*x2 - 1]


In [96]:
done = vcat(zero_ideal, univariate_ideal, principal_ideal)
done = [a[11:130] for a in done]
done2 = vcat(not_realizable, done)
unique(done2)    

148-element Vector{String}:
 "0******0******0***0************" ⋯ 58 bytes ⋯ "************0*******0*******0**"
 "0******0******0***0******0*****" ⋯ 58 bytes ⋯ "**************0****0********0**"
 "0******0******0***0******0*****" ⋯ 58 bytes ⋯ "***************0***0*******0***"
 "0******0******0***0******0*****" ⋯ 58 bytes ⋯ "**************0****0********0**"
 "0******0******0***0******0*****" ⋯ 58 bytes ⋯ "**************0*0***********0**"
 "0******0******0***0******0*****" ⋯ 58 bytes ⋯ "***************0***0***0*******"
 "0******0******0***0******0*****" ⋯ 58 bytes ⋯ "***************0**0*******0****"
 "0******0******0***0******0*****" ⋯ 58 bytes ⋯ "**************0******0*0*******"
 "0******0******0***0******0*****" ⋯ 58 bytes ⋯ "**************0***0*********0**"
 "0******0******0***0******0*****" ⋯ 58 bytes ⋯ "**************0******0***0*****"
 "0******0******0***0******0*****" ⋯ 58 bytes ⋯ "**************0******0****0****"
 "0******0******0***0******0*0***" ⋯ 58 bytes ⋯ "***************0**0**

In [98]:
rem = setdiff(lines_3_precomputed, done2)

3-element Vector{String}:
 "0******0******0**********0*****" ⋯ 58 bytes ⋯ "***************0**0****0*******"
 "0******0******0**********0*****" ⋯ 58 bytes ⋯ "**************0******0*0*******"
 "0******0******0**********0*****" ⋯ 58 bytes ⋯ "**************0******0***0*****"

In [108]:
Q = matroid_from_revlex_basis_encoding(rem[1], 3, 10)
As = optimal_circuits(Q)
MR = new_matroid_realization_space(Q,As[1];F=QQ,saturate=true)
MR2 = reduce_ideal_full(MR)
to_star0(As[1], 10) * rem[1]

"**00*000*00******0******0**********0****************0********************************0******0*********************0**0****0*******"

In [109]:
Q = matroid_from_revlex_basis_encoding(rem[2], 3, 10)
As = optimal_circuits(Q)
MR = new_matroid_realization_space(Q,As[1];F=QQ,saturate=true)
MR2 = reduce_ideal_full(MR)
to_star0(As[1], 10) * rem[2]

"**00*0000*0******0******0**********0****************0*************************0**************0*******************0******0*0*******"

In [110]:
Q = matroid_from_revlex_basis_encoding(rem[3], 3, 10)
As = optimal_circuits(Q)
MR = new_matroid_realization_space(Q,As[1];F=QQ,saturate=true)
MR2 = reduce_ideal_full(MR)
to_star0(As[1], 10) * rem[3]

"**00**00000******0******0**********0****************0***********************0****************0*******************0******0***0*****"

```
**00*000*00******0******0**********0****************0********************************0******0*********************0**0****0*******
**00*0000*0******0******0**********0****************0*************************0**************0*******************0******0*0*******
**00**00000******0******0**********0****************0***********************0****************0*******************0******0***0*****
```